In [ ]:
# block1
!pip install -q gspread beautifulsoup4 requests
# claude block2 addition
!pip install -q gspread beautifulsoup4 requests Pillow
!pip install -q gspread beautifulsoup4 requests Pillow numpy

In [ ]:
# block2
import gspread
import requests
from bs4 import BeautifulSoup
# claude
from PIL import Image
import numpy as np
from difflib import SequenceMatcher

print("✅ All packages loaded successfully")

✅ All packages loaded successfully


In [ ]:
# block3
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# block4
# Replace with your Sheet URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1TQNohr55e5NMspY2HZ1DfeFHkfZkG1vKQMs9GEeBrtM/edit'
spreadsheet = gc.open_by_url(sheet_url)
sheet = spreadsheet.sheet1

rows = sheet.get_all_records()

In [ ]:
#v10 Block5
import requests
from bs4 import BeautifulSoup
import re
from PIL import Image
import io
import os
import time
from urllib.parse import urlparse, urljoin, urlunsplit
from collections import defaultdict

def get_valid_images(site_url, max_images=50):
    """Enhanced image scraper with same-domain page crawling"""

    print(f"\n🔍 ENHANCED CRAWLING: {site_url}")
    print("="*60)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://google.com/'
    }

    all_images = set()
    pages_visited = set()
    start_time = time.time()

    try:
        # Step 1: Discover pages to crawl
        pages_to_crawl = discover_relevant_pages(site_url, headers)
        print(f"📄 Found {len(pages_to_crawl)} pages to crawl")

        # Step 2: Crawl each page for images
        for i, page_url in enumerate(pages_to_crawl):
            # Time limit check (3 minutes max per website)
            if time.time() - start_time > 180:
                print(f"⏰ Time limit reached, stopping at {i+1} pages")
                break

            # Image limit check
            if len(all_images) >= max_images:
                print(f"🖼️  Image limit reached ({max_images}), stopping")
                break

            if page_url not in pages_visited:
                print(f"\n📖 Crawling page {i+1}: {page_url}")
                page_images = scrape_images_from_page(page_url, headers)
                all_images.update(page_images)
                pages_visited.add(page_url)

                print(f"   Found {len(page_images)} images (Total: {len(all_images)})")

                # Respectful delay between pages
                time.sleep(2)

        # Step 3: Filter and validate images
        print(f"\n🔍 FILTERING {len(all_images)} unique images...")
        valid_images = filter_and_validate_images(all_images, headers, max_images)

        print(f"\n🎯 FINAL RESULT: {len(valid_images)} valid images from {len(pages_visited)} pages")
        return valid_images

    except Exception as e:
        print(f"❌ Error during enhanced crawling: {e}")
        return []

def discover_relevant_pages(base_url, headers):
    """Discover relevant pages on the same domain"""

    parsed_base = urlparse(base_url)
    base_domain = f"{parsed_base.scheme}://{parsed_base.netloc}"

    pages_to_crawl = [base_url]  # Start with homepage

    try:
        response = requests.get(base_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')

        # High-priority page patterns to look for
        priority_patterns = [
            'gallery', 'photos', 'images', 'pictures',
            'programs', 'projects', 'initiatives', 'work',
            'impact', 'stories', 'moments', 'portfolio',
            'about', 'team', 'staff', 'people',
            'news', 'blog', 'updates', 'events'
        ]

        # Find all links on the homepage
        links = soup.find_all('a', href=True)
        candidate_pages = set()

        for link in links:
            href = link.get('href')
            if href:
                # Make absolute URL
                full_url = urljoin(base_url, href)
                parsed_url = urlparse(full_url)

                # Only same domain
                if parsed_url.netloc == parsed_base.netloc:
                    # Check if URL matches priority patterns
                    url_path = parsed_url.path.lower()
                    link_text = link.get_text().lower()

                    for pattern in priority_patterns:
                        if pattern in url_path or pattern in link_text:
                            candidate_pages.add(full_url)
                            break

        # Add common page paths even if not found in links
        common_paths = [
            '/gallery/', '/gallery', '/photos/', '/photos',
            '/programs/', '/programs', '/projects/', '/projects',
            '/about/', '/about', '/team/', '/team',
            '/news/', '/news', '/blog/', '/blog',
            '/events/', '/events', '/impact/', '/impact'
        ]

        for path in common_paths:
            test_url = base_domain + path
            candidate_pages.add(test_url)

        # Limit to 15 pages max
        pages_to_crawl.extend(list(candidate_pages)[:14])  # 14 + homepage = 15 total

        return pages_to_crawl[:15]  # Safety limit

    except Exception as e:
        print(f"❌ Error discovering pages: {e}")
        return [base_url]  # Fallback to just homepage

def scrape_images_from_page(page_url, headers):
    """Scrape images from a single page"""

    page_images = set()

    try:
        response = requests.get(page_url, headers=headers, timeout=10)
        if response.status_code != 200:
            return page_images

        soup = BeautifulSoup(response.text, 'html.parser')

        # Method 1: Standard img tags with comprehensive srcset support
        img_tags = soup.find_all('img')
        for tag in img_tags:
            # Check multiple attributes including srcset
            for attr in ['src', 'data-src', 'data-lazy', 'data-original', 'data-load']:
                src = tag.get(attr)
                if src and not src.startswith('data:'):
                    full_url = make_absolute_url(src, page_url)
                    if full_url and is_image_url(full_url):
                        page_images.add(full_url)

            # ENHANCED: Parse srcset attribute for high-quality images
            srcset_attrs = ['srcset', 'data-srcset', 'data-lazy-srcset']
            for srcset_attr in srcset_attrs:
                srcset = tag.get(srcset_attr)
                if srcset:
                    srcset_urls = parse_srcset(srcset, page_url)
                    page_images.update(srcset_urls)
                    if srcset_urls:
                        print(f"   Found {len(srcset_urls)} images from {srcset_attr}")

        # Method 2: Look for gallery-specific containers
        gallery_containers = soup.select('[class*="gallery"], [class*="Gallery"], [id*="gallery"], [data-ux*="Gallery"]')
        for container in gallery_containers:
            container_imgs = container.find_all('img')
            for img in container_imgs:
                for attr in ['src', 'data-src', 'srcset', 'data-srcset']:
                    value = img.get(attr)
                    if value and not value.startswith('data:'):
                        if attr in ['srcset', 'data-srcset']:
                            srcset_urls = parse_srcset(value, page_url)
                            page_images.update(srcset_urls)
                        else:
                            full_url = make_absolute_url(value, page_url)
                            if full_url and is_image_url(full_url):
                                page_images.add(full_url)

            # ENHANCED: Parse srcset attribute for high-quality images
            srcset = tag.get('srcset')
            if srcset:
                srcset_urls = parse_srcset(srcset, page_url)
                page_images.update(srcset_urls)

        # Method 3: Background images in style attributes
        elements_with_style = soup.find_all(attrs={'style': True})
        for element in elements_with_style:
            style = element.get('style', '')
            if 'background-image:' in style or 'background:' in style:
                matches = re.findall(r'url\(["\']?([^"\'()]+)["\']?\)', style)
                for match in matches:
                    full_url = make_absolute_url(match, page_url)
                    if full_url and is_image_url(full_url):
                        page_images.add(full_url)

        return page_images

    except Exception as e:
        print(f"   ❌ Error scraping {page_url}: {e}")
        return page_images

def parse_srcset(srcset_value, base_url):
    """Parse srcset attribute and return high-quality image URLs"""
    srcset_urls = set()

    if not srcset_value:
        return srcset_urls

    # Split by comma to get individual src descriptors
    src_descriptors = [desc.strip() for desc in srcset_value.split(',')]

    for descriptor in src_descriptors:
        if not descriptor:
            continue

        # Split by space - first part is URL, rest are descriptors (width, pixel density)
        parts = descriptor.split()
        if not parts:
            continue

        url = parts[0]
        full_url = make_absolute_url(url, base_url)

        if full_url and is_image_url(full_url):
            # Extract width info for quality prioritization
            width_info = 0
            for part in parts[1:]:
                if 'w' in part:  # width descriptor like "740w"
                    try:
                        width_info = int(part.replace('w', ''))
                    except:
                        pass
                elif 'x' in part:  # pixel density like "2x"
                    try:
                        density = float(part.replace('x', ''))
                        width_info = int(density * 400)  # Assume base 400px for density calc
                    except:
                        pass

            # Only add high-quality images (width > 500px or if we can't determine, add anyway)
            if width_info == 0 or width_info > 500:
                srcset_urls.add(full_url)

    return srcset_urls
    """Convert relative URLs to absolute"""
    if not url or url.startswith('data:'):
        return None

    url = url.strip()

    if url.startswith('//'):
        return 'https:' + url
    elif url.startswith('/'):
        parsed = urlparse(base_url)
        return f"{parsed.scheme}://{parsed.netloc}" + url
    elif url.startswith('http'):
        return url
    else:
        return urljoin(base_url, url)

def is_image_url(url):
    """Check if URL is likely an image"""
    if not url:
        return False

    url_lower = url.lower()

    # Check for image file extensions
    if any(ext in url_lower for ext in ['.jpg', '.jpeg', '.png', '.webp', '.gif', '.svg']):
        return True

    # Check for image-related keywords
    if any(keyword in url_lower for keyword in ['image', 'img', 'photo', 'picture']):
        return True

    return False

def filter_and_validate_images(image_urls, headers, max_images):
    """Filter and validate images by downloading and checking content"""

    valid_images = []

    for i, img_url in enumerate(image_urls):
        if len(valid_images) >= max_images:
            break

        try:
            # Quick junk filter
            if is_likely_junk_url(img_url):
                print(f"❌ Junk URL: {img_url[:60]}...")
                continue

            # Download and validate
            img_response = requests.get(img_url, timeout=10, headers=headers)
            if img_response.status_code != 200:
                print(f"❌ Download failed ({img_response.status_code}): {img_url[:60]}...")
                continue

            content_size = len(img_response.content)

            # Size check (10KB to 10MB)
            if content_size < 10 * 1024:
                print(f"❌ Too small ({content_size/1024:.1f}KB): {img_url[:60]}...")
                continue

            if content_size > 10 * 1024 * 1024:
                print(f"❌ Too large ({content_size/(1024*1024):.1f}MB): {img_url[:60]}...")
                continue

            # Content validation
            if is_good_image_content(img_response.content):
                valid_images.append(img_url)
                print(f"✅ VALID ({content_size/1024:.1f}KB): {img_url[:60]}...")
            else:
                print(f"❌ Poor content: {img_url[:60]}...")

        except Exception as e:
            print(f"❌ Error validating {img_url[:60]}...: {e}")
            continue

    return valid_images

def is_likely_junk_url(url):
    """Quick filter for obviously junk URLs"""
    url_lower = url.lower()
    filename = os.path.basename(urlparse(url).path).lower()

    # Obvious junk patterns
    junk_patterns = [
        'spacer.', 'blank.', 'pixel.', '1x1.', 'loading.', 'spinner.',
        'arrow.', 'button.', 'icon-arrow', 'icon-close', 'icon-menu'
    ]

    for pattern in junk_patterns:
        if pattern in filename:
            return True

    # Very small social media icons
    social_patterns = ['facebook.', 'twitter.', 'instagram.', 'linkedin.']
    for pattern in social_patterns:
        if pattern in filename and any(size in filename for size in ['16', '24', '32']):
            return True

    return False

def is_good_image_content(img_content):
    """Validate image content quality"""
    try:
        img = Image.open(io.BytesIO(img_content))
        width, height = img.size

        # Basic size requirements
        if width < 50 or height < 50:
            return False

        # Skip extremely thin images (dividers)
        aspect_ratio = max(width, height) / min(width, height)
        if aspect_ratio > 12:
            return False

        # Skip mostly blank images
        if is_mostly_blank(img):
            return False

        return True

    except Exception:
        return False

def is_mostly_blank(img, threshold=0.95):
    """Check if image is mostly blank/white"""
    try:
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Sample check for speed
        sample_img = img.resize((20, 20))
        pixels = list(sample_img.getdata())

        white_pixels = sum(1 for r, g, b in pixels if r > 240 and g > 240 and b > 240)
        return (white_pixels / len(pixels)) > threshold

    except:
        return False

In [ ]:
#v9 Block5
import requests
from bs4 import BeautifulSoup
import re
from PIL import Image
import io
import os
import time
from urllib.parse import urlparse, urljoin, urlunsplit
from collections import defaultdict

def get_valid_images(site_url, max_images=50):
    """Enhanced image scraper with same-domain page crawling"""

    print(f"\n🔍 ENHANCED CRAWLING: {site_url}")
    print("="*60)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://google.com/'
    }

    all_images = set()
    pages_visited = set()
    start_time = time.time()

    try:
        # Step 1: Discover pages to crawl
        pages_to_crawl = discover_relevant_pages(site_url, headers)
        print(f"📄 Found {len(pages_to_crawl)} pages to crawl")

        # Step 2: Crawl each page for images
        for i, page_url in enumerate(pages_to_crawl):
            # Time limit check (3 minutes max per website)
            if time.time() - start_time > 180:
                print(f"⏰ Time limit reached, stopping at {i+1} pages")
                break

            # Image limit check
            if len(all_images) >= max_images:
                print(f"🖼️  Image limit reached ({max_images}), stopping")
                break

            if page_url not in pages_visited:
                print(f"\n📖 Crawling page {i+1}: {page_url}")
                page_images = scrape_images_from_page(page_url, headers)
                all_images.update(page_images)
                pages_visited.add(page_url)

                print(f"   Found {len(page_images)} images (Total: {len(all_images)})")

                # Respectful delay between pages
                time.sleep(2)

        # Step 3: Filter and validate images
        print(f"\n🔍 FILTERING {len(all_images)} unique images...")
        valid_images = filter_and_validate_images(all_images, headers, max_images)

        print(f"\n🎯 FINAL RESULT: {len(valid_images)} valid images from {len(pages_visited)} pages")
        return valid_images

    except Exception as e:
        print(f"❌ Error during enhanced crawling: {e}")
        return []

def discover_relevant_pages(base_url, headers):
    """Discover relevant pages on the same domain"""

    parsed_base = urlparse(base_url)
    base_domain = f"{parsed_base.scheme}://{parsed_base.netloc}"

    pages_to_crawl = [base_url]  # Start with homepage

    try:
        response = requests.get(base_url, headers=headers, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')

        # High-priority page patterns to look for
        priority_patterns = [
            'gallery', 'photos', 'images', 'pictures',
            'programs', 'projects', 'initiatives', 'work',
            'impact', 'stories', 'moments', 'portfolio',
            'about', 'team', 'staff', 'people',
            'news', 'blog', 'updates', 'events'
        ]

        # Find all links on the homepage
        links = soup.find_all('a', href=True)
        candidate_pages = set()

        for link in links:
            href = link.get('href')
            if href:
                # Make absolute URL
                full_url = urljoin(base_url, href)
                parsed_url = urlparse(full_url)

                # Only same domain
                if parsed_url.netloc == parsed_base.netloc:
                    # Check if URL matches priority patterns
                    url_path = parsed_url.path.lower()
                    link_text = link.get_text().lower()

                    for pattern in priority_patterns:
                        if pattern in url_path or pattern in link_text:
                            candidate_pages.add(full_url)
                            break

        # Add common page paths even if not found in links
        common_paths = [
            '/gallery/', '/gallery', '/photos/', '/photos',
            '/programs/', '/programs', '/projects/', '/projects',
            '/about/', '/about', '/team/', '/team',
            '/news/', '/news', '/blog/', '/blog',
            '/events/', '/events', '/impact/', '/impact'
        ]

        for path in common_paths:
            test_url = base_domain + path
            candidate_pages.add(test_url)

        # Limit to 15 pages max
        pages_to_crawl.extend(list(candidate_pages)[:14])  # 14 + homepage = 15 total

        return pages_to_crawl[:15]  # Safety limit

    except Exception as e:
        print(f"❌ Error discovering pages: {e}")
        return [base_url]  # Fallback to just homepage

def scrape_images_from_page(page_url, headers):
    """Scrape images from a single page"""

    page_images = set()

    try:
        response = requests.get(page_url, headers=headers, timeout=10)
        if response.status_code != 200:
            return page_images

        soup = BeautifulSoup(response.text, 'html.parser')

        # Method 1: Standard img tags
        img_tags = soup.find_all('img')
        for tag in img_tags:
            # Check multiple attributes
            for attr in ['src', 'data-src', 'data-lazy', 'data-original', 'data-load']:
                src = tag.get(attr)
                if src and not src.startswith('data:'):
                    full_url = make_absolute_url(src, page_url)
                    if full_url and is_image_url(full_url):
                        page_images.add(full_url)

        # Method 2: Background images in style attributes
        elements_with_style = soup.find_all(attrs={'style': True})
        for element in elements_with_style:
            style = element.get('style', '')
            if 'background-image:' in style or 'background:' in style:
                matches = re.findall(r'url\(["\']?([^"\'()]+)["\']?\)', style)
                for match in matches:
                    full_url = make_absolute_url(match, page_url)
                    if full_url and is_image_url(full_url):
                        page_images.add(full_url)

        return page_images

    except Exception as e:
        print(f"   ❌ Error scraping {page_url}: {e}")
        return page_images

def make_absolute_url(url, base_url):
    """Convert relative URLs to absolute"""
    if not url or url.startswith('data:'):
        return None

    url = url.strip()

    if url.startswith('//'):
        return 'https:' + url
    elif url.startswith('/'):
        parsed = urlparse(base_url)
        return f"{parsed.scheme}://{parsed.netloc}" + url
    elif url.startswith('http'):
        return url
    else:
        return urljoin(base_url, url)

def is_image_url(url):
    """Check if URL is likely an image"""
    if not url:
        return False

    url_lower = url.lower()

    # Check for image file extensions
    if any(ext in url_lower for ext in ['.jpg', '.jpeg', '.png', '.webp', '.gif', '.svg']):
        return True

    # Check for image-related keywords
    if any(keyword in url_lower for keyword in ['image', 'img', 'photo', 'picture']):
        return True

    return False

def filter_and_validate_images(image_urls, headers, max_images):
    """Filter and validate images by downloading and checking content"""

    valid_images = []

    for i, img_url in enumerate(image_urls):
        if len(valid_images) >= max_images:
            break

        try:
            # Quick junk filter
            if is_likely_junk_url(img_url):
                print(f"❌ Junk URL: {img_url[:60]}...")
                continue

            # Download and validate
            img_response = requests.get(img_url, timeout=10, headers=headers)
            if img_response.status_code != 200:
                print(f"❌ Download failed ({img_response.status_code}): {img_url[:60]}...")
                continue

            content_size = len(img_response.content)

            # Size check (10KB to 10MB)
            if content_size < 10 * 1024:
                print(f"❌ Too small ({content_size/1024:.1f}KB): {img_url[:60]}...")
                continue

            if content_size > 10 * 1024 * 1024:
                print(f"❌ Too large ({content_size/(1024*1024):.1f}MB): {img_url[:60]}...")
                continue

            # Content validation
            if is_good_image_content(img_response.content):
                valid_images.append(img_url)
                print(f"✅ VALID ({content_size/1024:.1f}KB): {img_url[:60]}...")
            else:
                print(f"❌ Poor content: {img_url[:60]}...")

        except Exception as e:
            print(f"❌ Error validating {img_url[:60]}...: {e}")
            continue

    return valid_images

def is_likely_junk_url(url):
    """Quick filter for obviously junk URLs"""
    url_lower = url.lower()
    filename = os.path.basename(urlparse(url).path).lower()

    # Obvious junk patterns
    junk_patterns = [
        'spacer.', 'blank.', 'pixel.', '1x1.', 'loading.', 'spinner.',
        'arrow.', 'button.', 'icon-arrow', 'icon-close', 'icon-menu'
    ]

    for pattern in junk_patterns:
        if pattern in filename:
            return True

    # Very small social media icons
    social_patterns = ['facebook.', 'twitter.', 'instagram.', 'linkedin.']
    for pattern in social_patterns:
        if pattern in filename and any(size in filename for size in ['16', '24', '32']):
            return True

    return False

def is_good_image_content(img_content):
    """Validate image content quality"""
    try:
        img = Image.open(io.BytesIO(img_content))
        width, height = img.size

        # Basic size requirements
        if width < 50 or height < 50:
            return False

        # Skip extremely thin images (dividers)
        aspect_ratio = max(width, height) / min(width, width)
        if aspect_ratio > 12:
            return False

        # Skip mostly blank images
        if is_mostly_blank(img):
            return False

        return True

    except Exception:
        return False

def is_mostly_blank(img, threshold=0.95):
    """Check if image is mostly blank/white"""
    try:
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # Sample check for speed
        sample_img = img.resize((20, 20))
        pixels = list(sample_img.getdata())

        white_pixels = sum(1 for r, g, b in pixels if r > 240 and g > 240 and b > 240)
        return (white_pixels / len(pixels)) > threshold

    except:
        return False

In [ ]:
#claude a3 Block6
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
import hashlib
import time
import requests
from collections import defaultdict
from urllib.parse import urlparse
from PIL import Image
import io
from difflib import SequenceMatcher

# Authenticate user
auth.authenticate_user()

# Google Drive API client
drive_service = build('drive', 'v3')

# ID of your master parent folder
PARENT_FOLDER_ID = "1aA2dPCkcvjfWS6XrpF-cKdpUGUmN_u8F"

# Global tracking for this session
uploaded_files_cache = defaultdict(set)  # folder_id -> set of hashes
folder_contents_cache = {}  # folder_id -> {hash: filename}

def get_or_create_folder(folder_name, parent_id):
    """Create or get existing folder"""
    query = f"name = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and '{parent_id}' in parents and trashed = false"
    response = drive_service.files().list(q=query, spaces='drive').execute()
    files = response.get('files', [])

    if files:
        return files[0]['id']

    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_id]
    }
    folder = drive_service.files().create(body=file_metadata, fields='id').execute()
    return folder.get('id')

def get_folder_contents(folder_id, force_refresh=False):
    """Get all files in folder with caching"""
    if folder_id not in folder_contents_cache or force_refresh:
        print(f"🔄 Refreshing folder contents for {folder_id}")
        query = f"'{folder_id}' in parents and trashed = false"
        results = drive_service.files().list(
            q=query,
            fields="files(id, name, size, md5Checksum)",
            pageSize=1000
        ).execute()

        folder_contents = {}
        for item in results.get('files', []):
            filename = item['name']
            # Extract hash from filename (format: name_hash.ext)
            if '_' in filename:
                hash_part = filename.split('_')[-1].split('.')[0]
                if len(hash_part) >= 6:
                    folder_contents[hash_part] = filename

            # Also store MD5 if available from Drive
            if 'md5Checksum' in item:
                folder_contents[item['md5Checksum']] = filename

        folder_contents_cache[folder_id] = folder_contents
        print(f"📁 Found {len(folder_contents)} existing files in folder")

    return folder_contents_cache[folder_id]

def similarity_ratio(s1, s2):
    """Calculate similarity ratio between two strings"""
    return SequenceMatcher(None, s1, s2).ratio()

def is_visually_similar_to_existing(img_content, folder_contents, filename):
    """Check if image is visually similar to existing images"""
    try:
        # Simple heuristic: if filename base is very similar
        base_name = filename.lower().replace('+', ' ').replace('%20', ' ')

        for existing_filename in folder_contents.values():
            existing_base = existing_filename.lower().replace('+', ' ').replace('%20', ' ')

            # Check if base names are very similar (accounting for URL encoding)
            if similarity_ratio(base_name, existing_base) > 0.8:
                return True

        return False
    except:
        return False

def is_duplicate_image(img_content, folder_id, filename):
    """Enhanced duplicate detection with multiple methods"""

    # Method 1: Content hash
    content_hash = hashlib.md5(img_content).hexdigest()
    short_hash = content_hash[:8]

    # Method 2: Check session cache first (fastest)
    if content_hash in uploaded_files_cache[folder_id]:
        print(f"⚠️ Duplicate detected in session cache: {filename}")
        return True, content_hash

    # Method 3: Check existing folder contents
    folder_contents = get_folder_contents(folder_id)

    # Check for exact hash match
    if content_hash in folder_contents or short_hash in folder_contents:
        print(f"⚠️ Duplicate detected in folder: {filename}")
        uploaded_files_cache[folder_id].add(content_hash)
        return True, content_hash

    # Method 4: Visual similarity check
    if is_visually_similar_to_existing(img_content, folder_contents, filename):
        print(f"⚠️ Visually similar image detected: {filename}")
        return True, content_hash

    # Method 5: Filename similarity (same base name)
    base_name = filename.split('.')[0].lower()
    for existing_hash, existing_filename in folder_contents.items():
        existing_base = existing_filename.split('_')[0].lower()
        if base_name == existing_base and len(base_name) > 5:
            print(f"⚠️ Similar filename detected: {filename} vs {existing_filename}")
            return True, content_hash

    return False, content_hash

def upload_image_with_better_duplication_check(img_url, folder_id):
    """Enhanced image upload with robust duplication checking"""
    try:
        # Download the image
        response = requests.get(img_url, timeout=10)
        if response.status_code != 200:
            print(f"❌ Failed to download: {img_url}")
            return None

        # Generate filename from URL
        parsed_url = urlparse(img_url)
        filename = os.path.basename(parsed_url.path)

        # Clean filename
        if not filename or '.' not in filename:
            content_type = response.headers.get('content-type', '')
            if 'jpeg' in content_type:
                ext = '.jpg'
            elif 'png' in content_type:
                ext = '.png'
            elif 'gif' in content_type:
                ext = '.gif'
            else:
                ext = '.jpg'
            filename = f"image_{hash(img_url) % 10000}{ext}"

        # Enhanced duplication check
        is_duplicate, file_hash = is_duplicate_image(response.content, folder_id, filename)

        if is_duplicate:
            return None  # Skip upload

        # Create unique filename with longer hash
        unique_filename = f"{os.path.splitext(filename)[0]}_{file_hash[:8]}{os.path.splitext(filename)[1]}"

        # Create temporary file
        temp_filename = f"/tmp/{unique_filename}"

        # Save image temporarily
        with open(temp_filename, 'wb') as f:
            f.write(response.content)

        # Upload to Drive
        media = MediaFileUpload(temp_filename, resumable=True)
        file_metadata = {
            'name': unique_filename,
            'parents': [folder_id]
        }

        uploaded = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id, webViewLink'
        ).execute()

        # Update caches
        uploaded_files_cache[folder_id].add(file_hash)
        folder_contents_cache[folder_id][file_hash[:8]] = unique_filename

        # Clean up temporary file
        if os.path.exists(temp_filename):
            os.remove(temp_filename)

        print(f"✅ Uploaded: {unique_filename}")
        return uploaded.get('webViewLink')

    except Exception as e:
        print(f"❌ Error uploading {img_url}: {e}")
        return None

def create_or_get_folder(folder_name):
    """Enhanced folder creation with cache management"""
    folder_id = get_or_create_folder(folder_name, PARENT_FOLDER_ID)

    # Refresh folder contents cache for this folder
    get_folder_contents(folder_id, force_refresh=True)

    return folder_id

In [ ]:
#claude a3 block7
for i, row in enumerate(rows):
    try:
        url = row['npo_url']
        print(f"\n🔄 Processing {i+1}/{len(rows)}: {url}")

        # Extract domain for folder name
        domain = re.sub(r'https?://(www\.)?', '', url).split('/')[0]
        folder_id = create_or_get_folder(domain)

        # Get filtered image URLs
        image_urls = get_valid_images(url)
        print(f"📸 Found {len(image_urls)} valid images after filtering")

        # Upload each image with enhanced duplication checking
        uploaded_count = 0
        duplicate_count = 0

        for img_url in image_urls:
            result = upload_image_with_better_duplication_check(img_url, folder_id)
            if result:  # Successfully uploaded
                uploaded_count += 1
            else:  # Skipped as duplicate
                duplicate_count += 1

        # Update folder link in spreadsheet
        folder_link = f"https://drive.google.com/drive/folders/{folder_id}"
        sheet.update_cell(i + 2, list(row.keys()).index('folder_link') + 1, folder_link)

        print(f"✅ Completed {url}")
        print(f"   📤 Uploaded: {uploaded_count} new images")
        print(f"   🔄 Skipped: {duplicate_count} duplicates")

        # Small delay to avoid API rate limits
        time.sleep(1)

    except Exception as e:
        print(f"❌ Error processing row {i+1}: {e}")
        continue

print(f"\n🎉 Finished processing all {len(rows)} websites!")
print("💡 Tip: Re-run the same script to add new images without duplicating existing ones.")


🔄 Processing 1/20: https://www.passatempoeducativo.org.br/
🔄 Refreshing folder contents for 1B6ZrCyyTiekaV_mJPw1vIUUCqyw0ybL4
📁 Found 0 existing files in folder

🔍 ENHANCED CRAWLING: https://www.passatempoeducativo.org.br/
📄 Found 15 pages to crawl

📖 Crawling page 1: https://www.passatempoeducativo.org.br/
   Found 42 images (Total: 42)

📖 Crawling page 2: https://www.passatempoeducativo.org.br/projects
   Found 0 images (Total: 42)

📖 Crawling page 3: https://www.passatempoeducativo.org.br/gallery
   Found 0 images (Total: 42)

📖 Crawling page 4: https://www.passatempoeducativo.org.br/programs/
   Found 0 images (Total: 42)

📖 Crawling page 5: https://www.passatempoeducativo.org.br/blog
   Found 0 images (Total: 42)

📖 Crawling page 6: https://www.passatempoeducativo.org.br/projects/
   Found 0 images (Total: 42)

📖 Crawling page 7: https://www.passatempoeducativo.org.br/impact/
   Found 0 images (Total: 42)

📖 Crawling page 8: https://www.passatempoeducativo.org.br/about
   Found 0

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (89853021 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


✅ VALID (516.8KB): https://www.passatempoeducativo.org.br/img/patro02.png...
❌ Too small (4.0KB): https://www.passatempoeducativo.org.br/img/iconMaps.png...
✅ VALID (34.8KB): https://www.passatempoeducativo.org.br/img/patro01.png...
❌ Too small (2.1KB): https://www.passatempoeducativo.org.br/img/facebook.png...
❌ Too small (1.6KB): https://www.passatempoeducativo.org.br/img/twitter.png...
❌ Too small (0.8KB): https://www.passatempoeducativo.org.br/img/espanhol.png...
✅ VALID (48.9KB): https://www.passatempoeducativo.org.br/img/patro03.png...
❌ Too small (7.5KB): https://www.passatempoeducativo.org.br/img/despesas.png...
❌ Too small (4.2KB): https://www.passatempoeducativo.org.br/img/calc2.png...
❌ Too small (9.9KB): https://www.passatempoeducativo.org.br/img/ONG2022.png...
✅ VALID (11.8KB): https://www.passatempoeducativo.org.br/img/principalCoracao....
✅ VALID (198.5KB): https://www.passatempoeducativo.org.br/img/apoienossaong.jpg...
❌ Too small (1.7KB): https://www.passatempoeducativ

In [ ]:
#debugging scrapping issue for a particular website
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, urljoin

def debug_website_scraping(site_url):
    """Debug version to understand why images aren't being found"""
    print(f"\n🔍 DEBUGGING: {site_url}")
    print("="*50)

    # Test different user agents and headers
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'
    ]

    for i, ua in enumerate(user_agents):
        print(f"\n--- Attempt {i+1} with User Agent {i+1} ---")

        headers = {
            'User-Agent': ua,
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

        try:
            response = requests.get(site_url, timeout=15, headers=headers)
            print(f"✅ Status Code: {response.status_code}")
            print(f"✅ Content Length: {len(response.content)} bytes")
            print(f"✅ Content Type: {response.headers.get('content-type', 'Unknown')}")

            if response.status_code == 200:
                # Parse the HTML
                soup = BeautifulSoup(response.text, 'html.parser')

                # Check page title
                title = soup.find('title')
                print(f"📄 Page Title: {title.text.strip() if title else 'No Title'}")

                # Count different types of image references
                print(f"\n📊 IMAGE ANALYSIS:")

                # Standard img tags
                img_tags = soup.find_all('img')
                print(f"   🖼️  Standard <img> tags: {len(img_tags)}")

                # Show first few img tags for debugging
                for i, img in enumerate(img_tags[:5]):
                    src = img.get('src', 'NO SRC')
                    alt = img.get('alt', 'NO ALT')
                    print(f"      {i+1}. src='{src[:60]}...' alt='{alt[:30]}...'")

                # Check for lazy loading attributes
                lazy_attrs = ['data-src', 'data-lazy', 'data-original']
                lazy_count = 0
                for attr in lazy_attrs:
                    lazy_imgs = soup.find_all('img', attrs={attr: True})
                    lazy_count += len(lazy_imgs)
                    if lazy_imgs:
                        print(f"   🔄 Lazy loading ({attr}): {len(lazy_imgs)} images")

                # Check for background images
                bg_images = 0
                elements_with_style = soup.find_all(attrs={'style': True})
                for element in elements_with_style:
                    style = element.get('style', '')
                    if 'background-image' in style or 'background:' in style:
                        bg_images += 1

                print(f"   🎨 Background images: {bg_images}")

                # Check for gallery-specific elements
                gallery_selectors = [
                    '.gallery', '.photo-gallery', '.image-gallery',
                    '[class*="gallery"]', '[class*="photo"]',
                    '.portfolio', '.showcase', '.slider'
                ]

                gallery_elements = 0
                for selector in gallery_selectors:
                    try:
                        elements = soup.select(selector)
                        if elements:
                            gallery_elements += len(elements)
                            print(f"   🖼️  Gallery elements ({selector}): {len(elements)}")
                    except:
                        continue

                # Check for JavaScript/JSON with image URLs
                scripts = soup.find_all('script')
                js_image_count = 0
                for script in scripts[:3]:  # Check first 3 scripts
                    if script.string:
                        # Look for image patterns in JavaScript
                        img_patterns = re.findall(r'\.(?:jpg|jpeg|png|gif|webp)', script.string, re.IGNORECASE)
                        js_image_count += len(img_patterns)

                print(f"   📜 JavaScript image references: {js_image_count}")

                # Try to extract actual image URLs
                print(f"\n🔗 EXTRACTING IMAGE URLS:")
                image_urls = set()

                # From img tags
                for img in img_tags:
                    src = img.get('src')
                    if src:
                        full_url = make_absolute_url_debug(src, site_url)
                        if full_url and is_valid_image_url_debug(full_url):
                            image_urls.add(full_url)

                print(f"   Found {len(image_urls)} potential image URLs")

                # Show first few URLs
                for i, url in enumerate(list(image_urls)[:3]):
                    print(f"      {i+1}. {url}")

                # Test downloading first image
                if image_urls:
                    test_url = list(image_urls)[0]
                    print(f"\n🧪 TESTING IMAGE DOWNLOAD:")
                    try:
                        img_response = requests.get(test_url, timeout=10, headers=headers)
                        print(f"   ✅ Download Status: {img_response.status_code}")
                        print(f"   ✅ Content Size: {len(img_response.content)} bytes")
                        print(f"   ✅ Content Type: {img_response.headers.get('content-type', 'Unknown')}")

                        if len(img_response.content) >= 10 * 1024:
                            print(f"   ✅ Size check PASSED (≥10KB)")
                        else:
                            print(f"   ❌ Size check FAILED (<10KB)")

                    except Exception as e:
                        print(f"   ❌ Download failed: {e}")

                break  # Success, no need to try other user agents

        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")
            continue

    print("\n" + "="*50)

def make_absolute_url_debug(url, base_url):
    """Debug version of URL conversion"""
    if not url:
        return None

    url = url.strip()

    if url.startswith('//'):
        return 'https:' + url
    elif url.startswith('/'):
        return base_url.rstrip('/') + url
    elif url.startswith('http'):
        return url
    elif url.startswith('data:'):
        return None
    else:
        return urljoin(base_url, url)

def is_valid_image_url_debug(img_url):
    """Debug version of URL validation"""
    if not img_url:
        return False

    url_lower = img_url.lower()
    valid_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.gif']

    return any(ext in url_lower for ext in valid_extensions)

# Test the problematic website
debug_website_scraping("https://vedanshifoundation.org/")


🔍 DEBUGGING: https://vedanshifoundation.org/

--- Attempt 1 with User Agent 1 ---
✅ Status Code: 200
✅ Content Length: 274728 bytes
✅ Content Type: text/html;charset=utf-8
📄 Page Title: vedanshifoundation.org

📊 IMAGE ANALYSIS:
   🖼️  Standard <img> tags: 25
      1. src='//img1.wsimg.com/isteam/stock/y6Edqxn/:/rs=w:600,h:300,cg:tr...' alt='NO ALT...'
      2. src='//img1.wsimg.com/isteam/ip/608fae06-17bb-47e1-994c-c77827dcd...' alt='NO ALT...'
      3. src='//img1.wsimg.com/isteam/ip/608fae06-17bb-47e1-994c-c77827dcd...' alt='NO ALT...'
      4. src='data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=...' alt='NO ALT...'
      5. src='data:image/gif;base64,R0lGODlhAQABAAD/ACwAAAAAAQABAAACADs=...' alt='NO ALT...'
   🎨 Background images: 0
   🖼️  Gallery elements ([class*="gallery"]): 1
   📜 JavaScript image references: 0

🔗 EXTRACTING IMAGE URLS:
   Found 2 potential image URLs
      1. https://img1.wsimg.com/isteam/ip/608fae06-17bb-47e1-994c-c77827dcdc6a/WhatsApp%20Image%20202